In [1]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
from collections import Counter

from sklearn.model_selection import train_test_split
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced

from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler

import matplotlib.pyplot as plt
%matplotlib inline

import lightgbm as lgb


In [2]:
df=pd.read_csv("https://vikas-bucket0809.s3.ca-central-1.amazonaws.com/train.csv")

In [3]:
df.drop(['policy_id','area_cluster'], axis=1,inplace=True)
df.head()

,policy_tenure,age_of_car,age_of_policyholder,population_density,make,segment,model,fuel_type,max_torque,max_power,...,is_brake_assist,is_power_door_locks,is_central_locking,is_power_steering,is_driver_seat_height_adjustable,is_day_night_rear_view_mirror,is_ecw,is_speed_alert,ncap_rating,is_claim
0,0.515874,0.05,0.644231,4990,1,A,M1,CNG,60Nm@3500rpm,40.36bhp@6000rpm,...,No,No,No,Yes,No,No,No,Yes,0,0
1,0.672619,0.02,0.375000,27003,1,A,M1,CNG,60Nm@3500rpm,40.36bhp@6000rpm,...,No,No,No,Yes,No,No,No,Yes,0,0
2,0.841110,0.02,0.384615,4076,1,A,M1,CNG,60Nm@3500rpm,40.36bhp@6000rpm,...,No,No,No,Yes,No,No,No,Yes,0,0
3,0.900277,0.11,0.432692,21622,1,C1,M2,Petrol,113Nm@4400rpm,88.50bhp@6000rpm,...,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,2,0
4,0.596403,0.11,0.634615,34738,2,A,M3,Petrol,91Nm@4250rpm,67.06bhp@5500rpm,...,No,Yes,Yes,Yes,No,Yes,Yes,Yes,2,0


In [4]:
sum=df.isna().sum()
sum

policy_tenure                       0
age_of_car                          0
age_of_policyholder                 0
population_density                  0
make                                0
segment                             0
model                               0
fuel_type                           0
max_torque                          0
max_power                           0
engine_type                         0
airbags                             0
is_esc                              0
is_adjustable_steering              0
is_tpms                             0
is_parking_sensors                  0
is_parking_camera                   0
rear_brakes_type                    0
displacement                        0
cylinder                            0
transmission_type                   0
gear_box                            0
steering_type                       0
turning_radius                      0
length                              0
width                               0
height      

In [5]:
# computing number of rows
rows = len(df.axes[0])
# computing number of columns
cols = len(df.axes[1])
print("Number of Rows: ", rows)
print("Number of Columns: ", cols)

Number of Rows:  58592
Number of Columns:  42


In [6]:
column_list = list(df.select_dtypes(include=['O']).columns)
column_list

['segment',
 'model',
 'fuel_type',
 'max_torque',
 'max_power',
 'engine_type',
 'is_esc',
 'is_adjustable_steering',
 'is_tpms',
 'is_parking_sensors',
 'is_parking_camera',
 'rear_brakes_type',
 'transmission_type',
 'steering_type',
 'is_front_fog_lights',
 'is_rear_window_wiper',
 'is_rear_window_washer',
 'is_rear_window_defogger',
 'is_brake_assist',
 'is_power_door_locks',
 'is_central_locking',
 'is_power_steering',
 'is_driver_seat_height_adjustable',
 'is_day_night_rear_view_mirror',
 'is_ecw',
 'is_speed_alert']

In [7]:
# labelEncoder present in scikitlearn library
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
for x in column_list:
    df[x] = le.fit_transform(df[x])


In [ ]:
df.head(20)

,policy_tenure,age_of_car,age_of_policyholder,population_density,make,segment,model,fuel_type,max_torque,max_power,...,is_brake_assist,is_power_door_locks,is_central_locking,is_power_steering,is_driver_seat_height_adjustable,is_day_night_rear_view_mirror,is_ecw,is_speed_alert,ncap_rating,is_claim
0,0.515874,0.05,0.644231,4990,1,0,0,0,5,2,...,0,0,0,1,0,0,0,1,0,0
1,0.672619,0.02,0.375000,27003,1,0,0,0,5,2,...,0,0,0,1,0,0,0,1,0,0
2,0.841110,0.02,0.384615,4076,1,0,0,0,5,2,...,0,0,0,1,0,0,0,1,0,0
3,0.900277,0.11,0.432692,21622,1,3,3,2,0,6,...,1,1,1,1,1,1,1,1,2,0
4,0.596403,0.11,0.634615,34738,2,0,4,2,8,5,...,0,1,1,1,0,1,1,1,2,0
5,1.018709,0.07,0.519231,13051,3,4,5,1,4,0,...,1,1,1,1,1,0,1,1,3,0
6,0.097992,0.16,0.403846,6112,4,2,6,1,3,7,...,0,1,1,1,0,0,1,1,5,0
7,0.509085,0.14,0.423077,8794,1,2,7,2,0,6,...,1,1,1,1,1,1,1,1,2,0
8,0.282394,0.07,0.298077,6112,3,4,5,1,4,0,...,1,1,1,1,1,0,1,1,3,0
9,0.566255,0.04,0.442308,17804,1,2,8,2,0,6,...,1,1,1,1,1,1,1,1,0,0


In [8]:
list(df.select_dtypes(include=['O']).columns)

[]

In [9]:
df.columns

Index(['policy_tenure', 'age_of_car', 'age_of_policyholder',
       'population_density', 'make', 'segment', 'model', 'fuel_type',
       'max_torque', 'max_power', 'engine_type', 'airbags', 'is_esc',
       'is_adjustable_steering', 'is_tpms', 'is_parking_sensors',
       'is_parking_camera', 'rear_brakes_type', 'displacement', 'cylinder',
       'transmission_type', 'gear_box', 'steering_type', 'turning_radius',
       'length', 'width', 'height', 'gross_weight', 'is_front_fog_lights',
       'is_rear_window_wiper', 'is_rear_window_washer',
       'is_rear_window_defogger', 'is_brake_assist', 'is_power_door_locks',
       'is_central_locking', 'is_power_steering',
       'is_driver_seat_height_adjustable', 'is_day_night_rear_view_mirror',
       'is_ecw', 'is_speed_alert', 'ncap_rating', 'is_claim'],
      dtype='object')

In [10]:
col_names=['population_density','displacement','length', 'width', 'height', 'gross_weight',]

In [11]:

minmax_scaler=MinMaxScaler()
df_scaled=df.copy()
features=df_scaled[col_names]

df_scaled[col_names]=minmax_scaler.fit_transform(features.values)

df_scaled

,policy_tenure,age_of_car,age_of_policyholder,population_density,make,segment,model,fuel_type,max_torque,max_power,...,is_brake_assist,is_power_door_locks,is_central_locking,is_power_steering,is_driver_seat_height_adjustable,is_day_night_rear_view_mirror,is_ecw,is_speed_alert,ncap_rating,is_claim
0,0.515874,0.05,0.644231,0.064260,1,0,0,0,5,2,...,0,0,0,1,0,0,0,1,0,0
1,0.672619,0.02,0.375000,0.365231,1,0,0,0,5,2,...,0,0,0,1,0,0,0,1,0,0
2,0.841110,0.02,0.384615,0.051764,1,0,0,0,5,2,...,0,0,0,1,0,0,0,1,0,0
3,0.900277,0.11,0.432692,0.291660,1,3,3,2,0,6,...,1,1,1,1,1,1,1,1,2,0
4,0.596403,0.11,0.634615,0.470987,2,0,4,2,8,5,...,0,1,1,1,0,1,1,1,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58587,0.355089,0.13,0.644231,0.116270,2,0,4,2,8,5,...,0,1,1,1,0,1,1,1,2,0
58588,1.199642,0.02,0.519231,0.102516,1,0,0,0,5,2,...,0,0,0,1,0,0,0,1,0,0
58589,1.162273,0.05,0.451923,0.470987,1,0,0,0,5,2,...,0,0,0,1,0,0,0,1,0,0
58590,1.236307,0.14,0.557692,0.116270,1,2,7,2,0,6,...,1,1,1,1,1,1,1,1,2,0


In [12]:
# Create our features
X=df_scaled.drop('is_claim',axis=1)
# Create our target
y = df_scaled['is_claim']

In [13]:
X.head()

,policy_tenure,age_of_car,age_of_policyholder,population_density,make,segment,model,fuel_type,max_torque,max_power,...,is_rear_window_defogger,is_brake_assist,is_power_door_locks,is_central_locking,is_power_steering,is_driver_seat_height_adjustable,is_day_night_rear_view_mirror,is_ecw,is_speed_alert,ncap_rating
0,0.515874,0.05,0.644231,0.064260,1,0,0,0,5,2,...,0,0,0,0,1,0,0,0,1,0
1,0.672619,0.02,0.375000,0.365231,1,0,0,0,5,2,...,0,0,0,0,1,0,0,0,1,0
2,0.841110,0.02,0.384615,0.051764,1,0,0,0,5,2,...,0,0,0,0,1,0,0,0,1,0
3,0.900277,0.11,0.432692,0.291660,1,3,3,2,0,6,...,1,1,1,1,1,1,1,1,1,2
4,0.596403,0.11,0.634615,0.470987,2,0,4,2,8,5,...,0,0,1,1,1,0,1,1,1,2


In [14]:
X.describe()

,policy_tenure,age_of_car,age_of_policyholder,population_density,make,segment,model,fuel_type,max_torque,max_power,...,is_rear_window_defogger,is_brake_assist,is_power_door_locks,is_central_locking,is_power_steering,is_driver_seat_height_adjustable,is_day_night_rear_view_mirror,is_ecw,is_speed_alert,ncap_rating
count,58592.000000,58592.000000,58592.000000,58592.000000,58592.000000,58592.000000,58592.000000,58592.000000,58592.000000,58592.000000,...,58592.000000,58592.000000,58592.000000,58592.000000,58592.000000,58592.000000,58592.000000,58592.000000,58592.000000,58592.000000
mean,0.611246,0.069424,0.469420,0.253444,1.763722,1.938644,4.659237,1.003448,3.288538,3.317057,...,0.350133,0.549171,0.724246,0.724246,0.979366,0.585251,0.380308,0.724246,0.993805,1.759950
std,0.414156,0.056721,0.122886,0.241457,1.136988,1.566329,3.197355,0.835104,2.440212,2.566569,...,0.477016,0.497581,0.446897,0.446897,0.142158,0.492683,0.485467,0.446897,0.078467,1.389576
min,0.002735,0.000000,0.288462,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.210250,0.020000,0.365385,0.079601,1.000000,0.000000,0.000000,0.000000,0.000000,2.000000,...,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000
50%,0.573792,0.060000,0.451923,0.116270,1.000000,2.000000,5.000000,1.000000,4.000000,2.000000,...,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1.000000,1.000000,2.000000
75%,1.039104,0.110000,0.548077,0.365231,3.000000,4.000000,7.000000,2.000000,5.000000,6.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,3.000000
max,1.396641,1.000000,1.000000,1.000000,5.000000,5.000000,10.000000,2.000000,8.000000,8.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,5.000000


In [15]:

X_train, X_test, y_train, y_test = train_test_split(X, y,train_size=0.9,stratify=y)

In [16]:

std_scaler=StandardScaler()
X_train_scaled=std_scaler.fit_transform(X_train)
X_test_scaled=std_scaler.fit_transform(X_test)

In [17]:
from imblearn.combine import SMOTEENN

smote_enn=SMOTEENN(random_state=1)
X_resampled,y_resampled=smote_enn.fit_resample(X_train_scaled,y_train)
Counter(y_resampled)


Counter({0: 32375, 1: 40963})

In [28]:
model = lgb.LGBMClassifier(n_estimators=12000
                           ,learning_rate=1
                           ,n_jobs=-1
                           ,max_depth=128
                           
                         
                           )

In [29]:
model.fit(X_resampled, y_resampled)

# Predicting the Target variable
y_pred = model.predict(X_test_scaled)
accuracy = model.score(X_test, y_test)
print(accuracy)
print(classification_report(y_test, y_pred))

0.6464163822525597
              precision    recall  f1-score   support

           0       0.95      0.29      0.45      5485
           1       0.07      0.75      0.12       375

    accuracy                           0.32      5860
   macro avg       0.51      0.52      0.28      5860
weighted avg       0.89      0.32      0.42      5860

